In [28]:
import sys
sys.path.insert(0, "../models")
from pfld import InvertedResidual
from ghostnet import GhostBottleneck, GhostModule, GhostNet, ghostnet, _make_divisible, ConvBnAct
from pfld import PFLDInference, conv_bn
import torch
import time
import torch.nn as nn
import math

# cfgs = [
#         # k, t, c, SE, s 
#         # stage1
#         [[3,  16,  16, 0, 2]],
#         # stage2
#         [[3,  48,  24, 0, 1]],
#         [[3,  72,  24, 0, 1]],
#         # stage3
#         [[5,  72,  40, 0.25, 1]],
#         [[5, 120,  40, 0.25, 1]],
#         # stage4
#         [[3, 240,  80, 0, 1]], #The original number of channels here is 80, but I change to 64 so that it fit to the AuxiliaryNet
#         [[3, 200,  80, 0, 2],
#          [3, 184,  80, 0, 1],
#          [3, 184,  80, 0, 1],
#          [3, 480, 112, 0.25, 1],
#          [3, 672, 112, 0.25, 1]
#         ],
#         # stage5
#         [[5, 672, 160, 0.25, 1]],
#         [[5, 960, 160, 0, 1],
#          [5, 960, 160, 0.25, 1],
#          [5, 960, 160, 0, 1],
#          [5, 960, 160, 0.25, 1]
#         ],

#         # final
#         # [[5, 320, 16, 0.25, 1]]
#     ]

# cfgs = [
#     # k, t,    c,se,s 
#     [[3, 64*2, 64, 0, 2]],

#     [[3, 64*2, 64, 0.25,1],
#      [3, 64*2, 64, 0.25,1],
#      [3, 64*2, 64, 0.25,1],
#      [3, 64*2, 64, 0.25,1]
#     ],

#     [[3, 64*2, 128, 0, 2]],

#     [[3, 128*4, 128, 0, 1],
#      [3, 128*4, 128, 0.25, 1],
#      [3, 128*4, 128, 0.25, 1],
#      [3, 128*4, 128, 0.25, 1],
#      [3, 128*4, 128, 0.25, 1],
#      [3, 128*4, 128, 0.25, 1],
#      [3, 128*2, 16, 0, 1],
#     ]

# ]


cfgs = [
        # k, t, c, SE, s 
        # stage1
        [[3,  16,  16, 0, 1],
         [3,  48,  24, 0, 2], #56x56
        ],

        # stage 2
        [[3,  72,  24, 0, 1],
         [5,  72,  40, 0.25, 2] # 28x28
        ],

        # stage 3
        [[5, 120,  40, 0.25, 1],
         [3, 240,  80, 0, 2]  #14x14
        ],

        # stage 4
        [[3, 200,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 480, 112, 0.25, 1],
         [3, 672, 112, 0.25, 1],
         [5, 672, 160, 0.25, 1]
        ],

        # stage5
        [[5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1],
         [5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1]
        ]
    ]

class CustomizedGhostNet(nn.Module):
    def __init__(self, cfgs, width=1.0, dropout=0.2):
        super(CustomizedGhostNet, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = cfgs
        self.dropout = dropout

        # building first layer
        output_channel = _make_divisible(16 * width, 4)
        self.conv_stem = nn.Conv2d(3, output_channel, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_channel)
        self.act1 = nn.ReLU(inplace=True)
        input_channel = output_channel

        # building inverted residual blocks
        first_6_stages = []  # This one used for another branch
        remaining_stages = []
        stages = []
        block = GhostBottleneck
        for i, cfg in enumerate(self.cfgs):
            layers = []
            for k, exp_size, c, se_ratio, s in cfg:
                output_channel = _make_divisible(c * width, 4)
                hidden_channel = _make_divisible(exp_size * width, 4)
                layers.append(block(input_channel, hidden_channel, output_channel, k, s,
                            se_ratio=se_ratio))
                input_channel = output_channel

            if i<=1:
                first_6_stages.append(nn.Sequential(*layers))
            else:
                remaining_stages.append(nn.Sequential(*layers))
                

        output_channel = _make_divisible(exp_size * width, 4)
        # output_channel = 16
        print(f"Input channel: {input_channel}. Output channel: {output_channel}")
        remaining_stages.append(nn.Sequential(ConvBnAct(input_channel, output_channel, 1)))
        
        self.begining_blocks = nn.Sequential(*first_6_stages)
        self.remaining_blocks = nn.Sequential(*remaining_stages)  # 960x14x14
        self.conv6 = ConvBnAct(output_channel, 16, 1)

        self.relu = nn.ReLU(inplace=True)

        self.conv7 = conv_bn(16, 32, 3, 2)  # [32, 7, 7]
        self.conv8 = nn.Conv2d(32, 128, 7, 1, 0)  # [128, 1, 1]
        self.bn8 = nn.BatchNorm2d(128)

        self.avg_pool1 = nn.AvgPool2d(14)
        self.avg_pool2 = nn.AvgPool2d(7)
        self.fc = nn.Linear(176, 196)

        # building last several layers


    def forward(self, x):
        x = self.conv_stem(x)
        print("Size after conv_steem:", x.shape)
        x = self.bn1(x)
        x = self.act1(x)
        features_for_auxiliarynet = self.begining_blocks(x)
        x = self.remaining_blocks(features_for_auxiliarynet)
        print("After remaiing blocks: ", x.shape)

        x = self.conv6(x)

        x1 = self.avg_pool1(x)
        x1 = x1.view(x1.size(0), -1)

        x = self.conv7(x)
        x2 = self.avg_pool2(x)

        x2 = x2.view(x2.size(0), -1)

        x3 = self.relu(self.conv8(x))

        x3 = x3.view(x1.size(0), -1)

        multi_scale = torch.cat([x1, x2, x3], 1)
        landmarks = self.fc(multi_scale)
    
        return features_for_auxiliarynet, landmarks


model = CustomizedGhostNet(cfgs, width=1)
model.eval()
input = torch.randn(1,3,112,112)
t1 = time.time()
fea, x = model(input)
print("Time: ", time.time()-t1)
print("Shape fea: ", fea.shape)
print("Remainig X: ", x.shape)

# len(model.state_dict().keys())
b = list(model.state_dict().keys())

b1 = list(filter(lambda i: "shortcut" in i , b))
len(b1)
# # len(b1)
# for c,i in enumerate(b):
#   print(i)
#   if c >=210:
#     break

Input channel: 160. Output channel: 960
Size after conv_steem: torch.Size([1, 16, 112, 112])
After remaiing blocks:  torch.Size([1, 960, 14, 14])
Time:  0.048256635665893555
Shape fea:  torch.Size([1, 80, 14, 14])
Remainig X:  torch.Size([1, 196])


60

In [4]:
ck = torch.load("../checkpoint_imagenet/state_dict_93.98.pth")

In [32]:
# model.state_dict()["begining_blocks.0.0.ghost1.primary_conv.0.weight"].data.copy_(ck["blocks.0.0.ghost1.primary_conv.0.weight"])


In [10]:
a = list(ck.keys())
a1 = list(filter(lambda i: "shortcut" in i ,a))
len(a1)
# for c, i in enumerate(a):
#     print(i)
#     if c>=200:
#         break
 
# print(a)

60

In [26]:
def find_corresponding_layer(l):
    map_dict  = {
    "begining_blocks.0" : "blocks.0",
    "begining_blocks.1" : "blocks.1",
    "begining_blocks.2" : "blocks.2",
    "begining_blocks.3" : "blocks.3",
    "begining_blocks.4" : "blocks.4",
    "begining_blocks.5" : "blocks.5",
    "remaining_blocks.0" : "blocks.6",
    "remaining_blocks.1" : "blocks.7",
    "remaining_blocks.2" : "blocks.8",
    "remaining_blocks.3" : "blocks.9",
    }

    for k,v in map_dict.items():
        l = l.replace(k,v)

    return l    




'blocks.7.0.se.conv_expand.weight'

In [34]:
our_model_layer_keys = list(model.state_dict().keys())
pretrained_layer_keys = list(ck.keys())

for l in  our_model_layer_keys:
    l1 = find_corresponding_layer(l)
    if l1 in pretrained_layer_keys:
        if ck[l1].data.shape == model.state_dict()[l].data.shape:
            
        else:
            print(f"{l}. shape:{model.state_dict()[l].data.shape}. {l1}. Shape: {ck[l1].data.shape}")

In [25]:
print(len(model.state_dict().keys()))
print(len(ck.keys()))

a = list(model.state_dict().keys())
b = list(ck.keys())
a = list(filter(lambda i: "ghost" not in i ,a))
b = list(filter(lambda i: "ghost" not in i ,b))

a1 = list(map(lambda i: ".".join(i.split(".")[3:]) ,a))
b1 = list(map(lambda i: ".".join(i.split(".")[3:]) ,b))

# a1 = list(set(a1))
for r, (i, j) in enumerate(zip(a, b)):
    if r <600:  
        print(i, "      ----------------      ", j)
        # print(model.state_dict()[i].data.shape, "      ----------------      ", ck[j].data.shape)
        # print("\n")


535
512
conv_stem.weight       ----------------       conv_stem.weight
bn1.weight       ----------------       bn1.weight
bn1.bias       ----------------       bn1.bias
bn1.running_mean       ----------------       bn1.running_mean
bn1.running_var       ----------------       bn1.running_var
bn1.num_batches_tracked       ----------------       bn1.num_batches_tracked
begining_blocks.0.0.conv_dw.weight       ----------------       blocks.1.0.conv_dw.weight
begining_blocks.0.0.bn_dw.weight       ----------------       blocks.1.0.bn_dw.weight
begining_blocks.0.0.bn_dw.bias       ----------------       blocks.1.0.bn_dw.bias
begining_blocks.0.0.bn_dw.running_mean       ----------------       blocks.1.0.bn_dw.running_mean
begining_blocks.0.0.bn_dw.running_var       ----------------       blocks.1.0.bn_dw.running_var
begining_blocks.0.0.bn_dw.num_batches_tracked       ----------------       blocks.1.0.bn_dw.num_batches_tracked
begining_blocks.0.0.shortcut.0.weight       ----------------      

In [146]:
j = 0 # index of pretrain model
for l in a:
    l1 = ".".join(l.split(".")[3:])
    
    for k in range(j, len(b)):
        k1 = ".".join(b[j].split(".")[3:])
        if l1==k1 and l1!="":
            print(l1, "----------------", k1)
            print(model.state_dict()[l].shape,"-----------", ck[b[k]].shape)
            
            break
        j = j+1

    # if r<200:
        # print(model.state_dict()[i].data.shape, "----------", ck[j].data.shape)


In [2]:



model = ghostnet(width=1)
model.eval()
# print(model)
input = torch.randn(1,3,112,112)

t1 = time.time()
y = model(input)
print("Time inference one input:", time.time()-t1)
print(y.size())

input = torch.randn(1, 3, 112, 112)
plfd_backbone = PFLDInference()
plfd_backbone.eval()
t1 = time.time()
features, landmarks = plfd_backbone(input)
print("Time: ", time.time()-t1)

Feature after ghotnet block:  torch.Size([1, 960, 4, 4])
Feature after global pool:  torch.Size([1, 960, 1, 1])
Feature after conv_head:  torch.Size([1, 1280, 1, 1])
Time inference one input: 0.038233280181884766
torch.Size([1, 1000])
Out1 shape: torch.Size([1, 64, 28, 28])
X1 shape: torch.Size([1, 16, 1, 1])
X2 shape: torch.Size([1, 32, 1, 1])
X3 shape: torch.Size([1, 128, 1, 1])
Time:  0.03032398223876953


In [ ]:
# Residual
w=64
h=64
stride=2
expand_ratio=1
res = InvertedResidual(w, h, stride, False, expand_ratio=6)
res

In [28]:
# Ghost
input_channel=64
output_channel=64
k=2
s=2
se_ratio=2
gho = GhostModule(input_channel, output_channel, kernel_size=1, ratio=2, dw_size=3, stride=2, relu=True)
gho

GhostModule(
  (primary_conv): Sequential(
    (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (cheap_operation): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

In [37]:
import torch.nn as nn

ghobotto = GhostBottleneck(64, 64, 64, dw_kernel_size=3,
                 stride=1, act_layer=nn.ReLU, se_ratio=0.)
ghobotto

GhostBottleneck(
  (ghost1): GhostModule(
    (primary_conv): Sequential(
      (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (cheap_operation): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (ghost2): GhostModule(
    (primary_conv): Sequential(
      (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Sequential()
    )
    (cheap_operation): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_run

In [1]:
 cfgs = [
        # k, t, c, SE, s 
        # stage1
        [[3,  16,  16, 0, 1]],
        # stage2
        [[3,  48,  24, 0, 2]],
        [[3,  72,  24, 0, 1]],
        # stage3
        [[5,  72,  40, 0.25, 2]],
        [[5, 120,  40, 0.25, 1]],
        # stage4
        [[3, 240,  80, 0, 2]],
        [[3, 200,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 480, 112, 0.25, 1],
         [3, 672, 112, 0.25, 1]
        ],
        # stage5
        [[5, 672, 160, 0.25, 2]],
        [[5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1],
         [5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1]
        ]
    ]

print(len(cfgs))

9
